In [1]:
# Example for one service - similar structure should apply to other services.
import logging
import sys
from spyne import Application, rpc, ServiceBase, Unicode, ComplexModel
from spyne.protocol.soap import Soap11
from spyne.server.wsgi import WsgiApplication
import json

from spyne.util.wsgi_wrapper import run_twisted
from suds.client import Client

class DictionnaryItem(ComplexModel):
    key = Unicode
    value = Unicode

class LoanDemandService(ServiceBase):
    @rpc(Unicode, _returns=Unicode)
    def loanDemand(ctx, request):
        try:
            clientInfo = Client("http://localhost:8000/ClientInformationExtractionService?wsdl")
            clientSolvency = Client("http://localhost:8002/SolvencyVerificationService?wsdl")
            clientDecision = Client("http://localhost:8008/EvaluationDecisionService?wsdl")

            response_info = clientInfo.service.extraire_informations(request)
            response_solvency = clientSolvency.service.solvencyVerif(response_info)
            final_response = clientDecision.service.evaluationScore(response_solvency)

            return final_response.value
        except Exception as e:
            logging.error(f"Error during service call: {str(e)}")
            return f"Erreur lors de l'appel au service : {str(e)}"

application = Application([LoanDemandService], tns='sarra.brahem.loanDemandService', in_protocol=Soap11(), out_protocol=Soap11())

if __name__ == '__main__':
    wsgi_app = WsgiApplication(application)
    sys.exit(run_twisted([(wsgi_app, b'LoanDemandService')], 8004))


SystemExit: 

C:\Users\21627\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
